In [1]:
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4

!echo "deb https://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update

!sudo apt-get install -y clickhouse-server clickhouse-client

!sudo service clickhouse-server start
!clickhouse-client

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20210119~18.04.1).
dirmngr is already the newest version (2.2.4-1ubuntu1.4).
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 1,692 B of archives.
After this operation, 154 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 apt-transport-https all 1.6.13 [1,692 B]
Fetched 1,692 B in 0s (4,727 B/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [2]:
!pip install clickhouse-driver

     |████████████████████████████████| 481kB 8.1MB/s 


In [3]:
from clickhouse_driver import Client
client = Client(host='localhost')
client.execute('SHOW DATABASES')

[('default',), ('system',)]

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
dateparser = lambda x: datetime.strptime(x, "%d.%m.%Y")
df = pd.read_csv('data.csv',sep=';',decimal=',',parse_dates = ['date'], date_parser=dateparser)

In [7]:
print(df.head())
print(df.info())

        date   product  amount  delta
0 2020-01-01  product1      10    5.0
1 2020-01-02  product2      15    7.5
2 2020-01-03  product3      20   10.0
3 2020-01-04  product4      25   12.5
4 2020-01-05  product5      30   15.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     9 non-null      datetime64[ns]
 1   product  9 non-null      object        
 2   amount   9 non-null      int64         
 3   delta    9 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 416.0+ bytes
None


In [8]:
client.execute('DROP DATABASE IF EXISTS db')

[]

In [9]:
client.execute('CREATE DATABASE db')

[]

In [10]:
client.execute('SHOW DATABASES')

[('db',), ('default',), ('system',)]

In [11]:
# Старый вариант settings={'use_numpy': True}
client = Client(host='localhost', user='default', port='9000', database='db',settings={'columnar': True})

In [12]:
client.execute('DROP TABLE IF EXISTS test')

[]

In [13]:
client.execute('CREATE TABLE test (date Date, product String, amount Int32, delta Float32) ENGINE = Memory')

[]

In [14]:
client.execute('SHOW TABLES FROM db')

[('test',)]

In [15]:
client.execute("INSERT INTO test VALUES", df.to_dict('records'))

9

In [16]:
def select_clickhouse(sql):
  return client.query_dataframe(sql)

In [17]:
sql = '''SELECT * 
         FROM test as t 
         ORDER BY t.date
         LIMIT 25'''

In [18]:
select_clickhouse(sql)

,date,product,amount,delta
0,2020-01-01,product1,10,5.0
1,2020-01-02,product2,15,7.5
2,2020-01-03,product3,20,10.0
3,2020-01-04,product4,25,12.5
4,2020-01-05,product5,30,15.0
5,2020-01-06,product6,35,17.5
6,2020-01-07,product7,40,20.0
7,2020-01-08,product8,45,22.5
8,2020-01-09,product9,50,25.0
